In [ ]:
import pandas as pd
import numpy as np
import zipfile
import os
import re

SEARCH_TERM = "Eventos/Sinistros"
FILES = ["1T2023.csv", "2t2023.csv", "3T2023.csv", "4T2023.csv",
         "1T2024.csv", "2T2024.csv", "3T2024.csv",  "4T2024.csv",
         "1T2025.csv", "2T2025.csv", "3T2025.csv"]

def load_and_normalize(file_path):
    """Identifica o delimitador, carrega o arquivo e normaliza colunas."""
    try:

        df = pd.read_csv(file_path, sep=None, engine='python', encoding='utf-8-sig')
    except Exception:

        df = pd.read_csv(file_path, sep=';', encoding='iso-8859-1')


    df.columns = [c.strip().upper() for c in df.columns]


    df_filtered = df[df['DESCRICAO'].str.contains(SEARCH_TERM, case=False, na=False)].copy()


    df_filtered['DATA'] = pd.to_datetime(df_filtered['DATA'], errors='coerce')
    df_filtered['ANO'] = df_filtered['DATA'].dt.year
    df_filtered['TRIMESTRE'] = df_filtered['DATA'].dt.quarter

    return df_filtered

def mock_api_enrichment(reg_ans):
    """Simula uma integração com API para buscar CNPJ e Razão Social via Registro ANS."""

    mapping = {
        "360783": ("00.111.222/0001-99", "OPERADORA ALFA SA"),
        "408794": ("33.444.555/0001-88", "SAUDE BETA LTDA"),
    }
    return mapping.get(str(reg_ans), (f"CNPJ_{reg_ans}", f"RAZAO_{reg_ans}"))

consolidated_data = []

for f in FILES:
    if os.path.exists(f):
        temp_df = load_and_normalize(f)
        for _, row in temp_df.iterrows():
            cnpj, razao = mock_api_enrichment(row['REG_ANS'])
            consolidated_data.append({
                'CNPJ': cnpj,
                'RazaoSocial': razao,
                'Trimestre': row['TRIMESTRE'],
                'Ano': row['ANO'],
                'ValorDespesas': row['VL_SALDO_FINAL']
            })

df_final = pd.DataFrame(consolidated_data)

def clean_numeric(value):
    """Converte valores com vírgula ou strings para float purificado."""
    if pd.isna(value):
        return 0.0
    if isinstance(value, str):

        value = value.replace('.', '').replace(',', '.')
    try:
        return float(value)
    except ValueError:
        return 0.0

consolidated_data = []
for f in FILES:
    if os.path.exists(f):
        temp_df = load_and_normalize(f)
        for _, row in temp_df.iterrows():
            cnpj, razao = mock_api_enrichment(row['REG_ANS'])


            valor_limpo = clean_numeric(row['VL_SALDO_FINAL'])

            consolidated_data.append({
                'CNPJ': cnpj,
                'RazaoSocial': razao,
                'Trimestre': row['TRIMESTRE'],
                'Ano': row['ANO'],
                'ValorDespesas': valor_limpo
            })

df_final = pd.DataFrame(consolidated_data)


df_final['Status'] = np.where(df_final['ValorDespesas'] <= 0, 'Suspeito/Zerado', 'OK')


df_final.to_csv('consolidado_despesas.csv', index=False, sep=';', encoding='utf-8-sig')

with zipfile.ZipFile('consolidado_despesas.zip', 'w') as z:
    z.write('consolidado_despesas.csv')

print("Sucesso! O arquivo consolidado_despesas.zip foi gerado.")

Sucesso! O arquivo consolidado_despesas.zip foi gerado.
